# SemaDB | SemaDB

> [SemaFind](https://www.semafind.com) が提供する [SemaDB](https://www.semafind.com/products/semadb) は、AIアプリケーション構築のための手間のかからないベクトル類似性データベースです。提供される `SemaDB Cloud` は、手間をかけずにすぐに始められる開発者向けの体験を提供します。
>
> > [SemaDB](https://www.semafind.com/products/semadb) from [SemaFind](https://www.semafind.com) is a no fuss vector similarity database for building AI applications. The hosted `SemaDB Cloud` offers a no fuss developer experience to get started.

APIの完全なドキュメント、例、そしてインタラクティブなプレイグラウンドは、[RapidAPI](https://rapidapi.com/semafind-semadb/api/semadb)で利用可能です。

> The full documentation of the API along with examples and an interactive playground is available on [RapidAPI](https://rapidapi.com/semafind-semadb/api/semadb).

このノートブックは、`SemaDB Cloud` ベクターストアの使用方法を示しています。

> This notebook demonstrates usage of the `SemaDB Cloud` vector store.




## Load document embeddings | ドキュメントの埋め込みを読み込む

ローカルで実行するために、私たちは一般的に文章の埋め込みに使用される[Sentence Transformers](https://www.sbert.net/)を使用しています。LangChainが提供するどの埋め込みモデルでも使用できます。

> To run things locally, we are using [Sentence Transformers](https://www.sbert.net/) which are commonly used for embedding sentences. You can use any embedding model LangChain offers.




In [ ]:
!pip install sentence_transformers

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(len(docs))

114


## Connect to SemaDB | SemaDBに接続する

SemaDB Cloudは、認証に[RapidAPIのキー](https://rapidapi.com/semafind-semadb/api/semadb)を使用します。無料のRapidAPIアカウントを作成することで、あなたのキーを取得できます。

> SemaDB Cloud uses [RapidAPI keys](https://rapidapi.com/semafind-semadb/api/semadb) to authenticate. You can obtain yours by creating a free RapidAPI account.




In [4]:
import getpass
import os

os.environ["SEMADB_API_KEY"] = getpass.getpass("SemaDB API Key:")

SemaDB API Key: ········


In [5]:
from langchain.vectorstores import SemaDB
from langchain.vectorstores.utils import DistanceStrategy

SemaDBベクトルストアへのパラメータは、APIを直接反映しています：

> The parameters to the SemaDB vector store reflect the API directly:

* "mycollection"は、これらのベクトルを保存するコレクションの名前です。

  > "mycollection": is the collection name in which we will store these vectors.

* 768はベクトルの次元数です。私たちのケースでは、センテンストランスフォーマーの埋め込みによって768次元のベクトルが生成されます。

  > 768: is dimensions of the vectors. In our case, the sentence transformer embeddings yield 768 dimensional vectors.

* API\_KEY: これはあなたのRapidAPIキーです。

  > API\_KEY: is your RapidAPI key.

* embeddings: ドキュメント、テキスト、クエリの埋め込みがどのように生成されるかを示します。

  > embeddings: correspond to how the embeddings of documents, texts and queries will be generated.

* DistanceStrategy: 使用される距離メトリックです。COSINEが使用される場合、ラッパーは自動的にベクトルを正規化します。

  > DistanceStrategy: is the distance metric used. The wrapper automatically normalises vectors if COSINE is used.





In [6]:
db = SemaDB("mycollection", 768, embeddings, DistanceStrategy.COSINE)

# Create collection if running for the first time. If the collection
# already exists this will fail.
db.create_collection()

True

SemaDBベクトルストアラッパーは、後で収集するためにドキュメントテキストをポイントメタデータとして追加します。大量のテキストを保存することは*推奨されていません*。大規模なコレクションをインデックス化する場合、外部IDなどドキュメントへの参照を保存することを推奨します。

> The SemaDB vector store wrapper adds the document text as point metadata to collect later. Storing large chunks of text is *not recommended*. If you are indexing a large collection, we instead recommend storing references to the documents such as external Ids.




In [7]:
db.add_documents(docs)[:2]

['813c7ef3-9797-466b-8afa-587115592c6c',
 'fc392f7f-082b-4932-bfcc-06800db5e017']

## Similarity Search | 類似性検索

私たちは、最も類似した文を検索するために、デフォルトのLangChain類似性検索インターフェイスを使用しています。

> We use the default LangChain similarity search interface to search for the most similar sentences.




In [8]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [9]:
docs = db.similarity_search_with_score(query)
docs[0]

(Document(page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../modules/state_of_the_union.txt', 'text': 'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'}),
 0.42369342)

## Clean up | クリーンアップ

コレクションを削除することで、すべてのデータを削除できます。

> You can delete the collection to remove all data.




In [10]:
db.delete_collection()

True